In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with A2A on Agent Engine

Adopted from the [Vertex AI Agent Engine Quickstart](https://cloud.google.com/vertex-ai/docs/agent-engine/quickstart)

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/agents/agent_engine/tutorial_a2a_on_agent_engine.ipynb 


## Overview

This notebook shows how to build, deploy, and interact with **[Agent2Agent (A2A) protocol](https://a2aprotocol.org)** agents hosted on the fully-managed, serverless **Vertex AI Agent Engine**.

A2A is an open standard, like HTTP for AI agents, enabling communication and collaboration between diverse AI agents by standardizing capability discovery (via Agent Cards) and interaction for complex tasks, thereby eliminating custom integrations.

Vertex AI Agent Engine is fully-managed, serverless platform for running A2A agents. It handles all the infrastructure, scaling, security, and monitoring so you can focus on your agent's logic.

In this tutorial, you will:

* **Build** a simple, A2A-compliant agent using the Vertex AI SDK.  
* **Test** the agent locally to ensure it works as expected.  
* **Deploy** the agent to Agent Engine with a single command.  
* **Query** the managed agent endpoint using three different methods (Vertex AI SDK, A2A SDK, and direct HTTP requests). 
* **Clean up** the resources you've created.

## Get started

### Install required packages

First, we'll install the necessary packages.

- `a2a-sdk` is the foundational open-source SDK for building A2A-compliant agents.
- `google-cloud-aiplatform` is the Vertex AI SDK, containing the new Agent Engine template we'll use for deployment.

In [ ]:
%pip install --upgrade --quiet "a2a-sdk>=0.3.4" --force-reinstall --quiet
%pip install --upgrade --quiet "google-cloud-aiplatform[agent_engines, adk]>=1.112.0" --force-reinstall --quiet

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
# import sys
# 
# if "google.colab" in sys.modules:
#     from google.colab import auth
# 
#     auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai
from google.genai import types
from dotenv import load_dotenv
load_dotenv()  # 


PROJECT_ID = os.environ.get("PROJECT_ID")  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

BUCKET_NAME = f"{PROJECT_ID}-bucket"  # @param {type: "string", placeholder: "[your-bucket-name]", isTemplate: true}
if not BUCKET_NAME or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID

BUCKET_URI = f"gs://{BUCKET_NAME}"

# !gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

# Initialize Vertex AI session
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

# Initialize the Gen AI client using http_options
# The parameter customizes how the Vertex AI client communicates with Google Cloud's backend services.
# It's used here to access new, pre-release features.
client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=types.HttpOptions(
        api_version="v1beta1", base_url=f"https://{LOCATION}-aiplatform.googleapis.com/"
    ),
)

In [ ]:
PROJECT_NUMBER = os.environ.get("PROJECT_NUMBER")
MCP_SERVER_URL = os.environ.get("MCP_SERVER_URL")
print(f"PROJECT_NUMBER: {PROJECT_NUMBER}")

### Import libraries

Here, we're importing all the necessary Python classes and functions we'll use throughout the notebook.

In [ ]:
# Helpers
import json
import logging
import time
from collections.abc import Awaitable, Callable
from datetime import datetime
from pprint import pprint
from typing import Any, NoReturn

import httpx
from IPython.display import Markdown, display
from google.auth import default
from google.auth.transport.requests import Request as req
from starlette.requests import Request

logging.getLogger().setLevel(logging.INFO)


# A2A
from a2a.client import ClientConfig, ClientFactory
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import TaskUpdater
from a2a.types import (
    AgentSkill,
    Message,
    Part,
    Role,
    TaskState,
    TaskQueryParams,
    TextPart,
    TransportProtocol,
    UnsupportedOperationError,
)
from a2a.utils import new_agent_text_message
from a2a.utils.errors import ServerError

# ADK
from google.adk import Runner
from google.adk.agents import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search_tool
from google.genai import types

# Agent Engine
from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card

### Helpers

These are simple utility functions to make our lives easier, especially for local testing. They help create mock HTTP requests (`build_post_request`, `build_get_request`) and fetch authentication tokens (`get_bearer_token`).

In [ ]:
def receive_wrapper(data: dict) -> Callable[[], Awaitable[dict]]:
    """Creates a mock ASGI receive callable for testing."""

    async def receive():
        byte_data = json.dumps(data).encode("utf-8")
        return {"type": "http.request", "body": byte_data, "more_body": False}

    return receive


def build_post_request(
    data: dict[str, Any] | None = None, path_params: dict[str, str] | None = None
) -> Request:
    """Builds a mock Starlette Request object for a POST request with JSON data."""
    scope = {
        "type": "http",
        "http_version": "1.1",
        "headers": [(b"content-type", b"application/json")],
        "app": None,
    }
    if path_params:
        scope["path_params"] = path_params
    receiver = receive_wrapper(data)
    return Request(scope, receiver)


def build_get_request(path_params: dict[str, str]) -> Request:
    """Builds a mock Starlette Request object for a GET request."""
    scope = {
        "type": "http",
        "http_version": "1.1",
        "query_string": b"",
        "app": None,
    }
    if path_params:
        scope["path_params"] = path_params

    async def receive():
        return {"type": "http.disconnect"}

    return Request(scope, receive)


def get_bearer_token() -> str | None:
    """Fetches a Google Cloud bearer token using Application Default Credentials."""
    try:
        # Use an alias to avoid name collision with starlette.requests.Request
        credentials, project = default(
            scopes=["https://www.googleapis.com/auth/cloud-platform"]
        )
        request = req()
        credentials.refresh(request)
        return credentials.token
    except Exception as e:
        print(f"Error getting credentials: {e}")
        print(
            "Please ensure you have authenticated with 'gcloud auth application-default login'."
        )
    return None

### Build a simple ADK agent

Before we can build an A2A agent, we need an agent. We create an agent using the Agent Development Kit (ADK).
The agent will be used by the agent executor to respond to user queries.

### Define the agent card

AgentCard is an important component of the A2A protocol. Think of it as a digital business card for your agent. It's a structured JSON document that tells other agents everything they need to know to interact with yours: its name, what it does, the skills it offers, and how to call its API endpoint.

We define an `AgentSkill` to describe our agent's Q&A capability. Then, we use the `create_agent_card` helper function to assemble the full card, including the agent's name, description, and the skill we just defined.

> Note: The utility builds the card based on the limitations in the current integration: Streaming is turned off and supports Authenticated Extended Card is turned on. Also `create_agent_card` supports `agent_card` which allows you to supply an `agent_card` as dictionary. If an Agent Card is supplied as a dictionary, validation errors might show depending on whether the card meets the current integration limitations.

Let's print the AgentCard we just created.

Take a look at the structure. You can see key fields like name, description, skills, and the url. For now, the URL points to localhost, which is perfect for local testing. When we deploy to Agent Engine, this URL will be automatically updated to point to the managed endpoint.


In [ ]:
from cocktail_agent_card import cocktail_agent_card

In [ ]:
cocktail_agent_card.model_dump()

### Define the agent executor

The AgentExecutor is the bridge between the A2A protocol and our agent's internal logic. It's a class that you implement to handle incoming A2A requests. It has two main methods:

*   `execute`: This is the main entry point. When a message arrives, this method gets the user's query from the RequestContext, creates a TaskUpdater - a handy A2A SDK tool for managing the task's lifecycle (e.g., setting its state to working), calls the ADK Runner to process the query with the Gemini model and Google Search tool, asynchronously waits for the final response from the agent, packages the text response into an A2A Artifact—the official output of a task and finally, marks the task as completed.
*   `cancel`: Our simple agent doesn't support long-running, cancelable jobs, so we simply state that the operation is unsupported.


In [ ]:
from cocktail_agent_executor2 import CocktailAgentExecutor

### Test the agent locally

Before deploying anything to the cloud, a crucial step is to test locally. This allows for rapid iteration and debugging.

The A2aAgent class from the Vertex AI SDK is our deployable unit. It wraps our AgentCard and AgentExecutor together. Calling set_up() prepares an in-memory server, allowing us to simulate calls to the agent as if it were deployed.


In [ ]:
a2a_agent = A2aAgent(agent_card=cocktail_agent_card, agent_executor_builder=CocktailAgentExecutor)
a2a_agent.set_up()

#### Get the agent card

At this point, we can call the handle_authenticated_agent_card method on our local agent instance to simulate a client discovering our agent by requesting its "business card." It would return the agent's capabilities, skills, and its endpoint URL, confirming our local server is set up correctly.

In [ ]:
request = build_get_request(None)
response = await a2a_agent.handle_authenticated_agent_card(
    request=request, context=None
)
pprint(response)

#### Send a query

Finally, let's call the on_message_send method, which is the A2A endpoint for starting a new task.

The agent immediately responds with a Task object in the TASK_STATE_SUBMITTED state. This is standard asynchronous behavior: the system acknowledges the request and gives us a task_id to track its progress.


In [ ]:
message_data = {
    "message": {
        "messageId": f"msg-{os.urandom(8).hex()}",
        "content": [{"text": "ingredients for a Margarita?"}],
        "role": "ROLE_USER",
    },
}
request = build_post_request(message_data)
response = await a2a_agent.on_message_send(request=request, context=None)
pprint(response)

We simply extract the task_id from the previous response and print it. We'll use this ID in the next step to fetch our answer.


In [ ]:
task_id = response["task"]["id"]
print(f"The Task ID is: {task_id}")

#### Get the response

With the task_id in hand, we can now poll for the result. We call the on_get_task method, which retrieves the current status of our task.

Since our ADK agent is fast, the task should have already moved to the
TASK_STATE_COMPLETED state. Notice the artifacts field in the response. This contains the answer to our question, neatly packaged as a text part.

In [ ]:
task_data = {"id": task_id}
request = build_get_request(task_data)
response = await a2a_agent.on_get_task(request=request, context=None)
print(response)
for artifact in response["artifacts"]:
    # Access the text through the 'root' attribute of the Part object
    if artifact["parts"] and "text" in artifact["parts"][0]:
        display(Markdown(f"**Answer**:\n {artifact['parts'][0]['text']}"))
    else:
        print("Could not extract text from artifact parts.")

#### (Optional) Cancel a task

If your agent executes long run operations, you can always cancel the associated task as shown below:

```py
# Local agent
task_id = response['task']['id']
task_data={"id": task_id}
request = build_post_request(path_params=task_data)
response = await long_running_agent.on_cancel_task(request=request, context=None)
```

### Deploy on Agent Engine

Now it is time to deploy the agent to a fully-managed, scalable platform, Vertex AI Agent Engine.

With a single agent_engines.create() call, the Vertex AI SDK performs a series of actions behind the scenes that allows you to scale your A2A agent. In order:

*   It takes our local `a2a_agent` object.
*   It serializes (pickles) the agent's code and its configuration.
*   It inspects the environment to determine the necessary Python package requirements.
*   It packages everything up and uploads it to the Cloud Storage bucket we configured earlier.
*   It provisions a secure, scalable, and fully-managed serverless endpoint on Agent Engine to host our agent.

In [ ]:
remote_a2a_agent = client.agent_engines.create(
    # The actual agent to deploy
    agent=a2a_agent,
    config={
        # Display name shown in the console
        "display_name": a2a_agent.agent_card.name,
        # Description for documentation
        "description": a2a_agent.agent_card.description,
        # Python dependencies needed in Agent Engine
        "service_account": f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com",
        "requirements": [
            "google-cloud-aiplatform[agent_engines,adk]>=1.112.0",
            "a2a-sdk >= 0.3.4",
            "pydantic==2.11.9",
            "cloudpickle==3.1.1",
            "PyJWT>=2.8.0",
            "jwt>=1.4.0",
            
        ],
        # Http options
        "http_options": {
            "base_url": f"https://{LOCATION}-aiplatform.googleapis.com",
            "api_version": "v1beta1",
        },
        # Staging bucket
        "staging_bucket": BUCKET_URI,
        "env_vars": {
            "COCKTAIL_REMOTE_MCP_SERVER_NAME": "cocktail-remote-mcp-server",
            "MCP_SERVER_URL": f"{MCP_SERVER_URL}",
        },
        "extra_packages": ["cocktail_agent_card.py","cocktail_agent_executor2.py"]
    },
)

### Get the remote agent card

The SDK handles the authentication and API call to our deployed endpoint, and we get back the AgentCard. The get method allows you to reconnect to an existing, already-deployed agent in a new session just by using its resource name.

Notice that the url field in the card now points to the public `aiplatform.googleapis.com` endpoint, not localhost.

In [ ]:
remote_a2a_agent_resource_name = remote_a2a_agent.api_resource.name
config = {"http_options": {"base_url": f"https://{LOCATION}-aiplatform.googleapis.com"}}

In [ ]:
remote_a2a_agent_resource_name = remote_a2a_agent.api_resource.name
config = {"http_options": {"base_url": f"https://{LOCATION}-aiplatform.googleapis.com"}}

remote_a2a_agent = client.agent_engines.get(
    name=remote_a2a_agent_resource_name,
    config=config,
)

remote_a2a_agent_card = await remote_a2a_agent.handle_authenticated_agent_card()
print(f"Agent: {remote_a2a_agent_card.name}")
print(f"URL: {remote_a2a_agent_card.url}")
print(f"Skills: {[s.description for s in remote_a2a_agent_card.skills]}")
print(f"Examples: {[s.examples for s in remote_a2a_agent_card.skills][0]}")

In [ ]:
#remote_a2a_agent_resource_name = remote_a2a_agent.api_resource.name
config = {"http_options": {"base_url": f"https://{LOCATION}-aiplatform.googleapis.com"}}

remote_agent = client.agent_engines.get(
    name=remote_a2a_agent_resource_name,
    config=config,
)

remote_a2a_agent_card = await remote_a2a_agent.handle_authenticated_agent_card()
print(f"Agent: {remote_a2a_agent_card.name}")
print(f"URL: {remote_a2a_agent_card.url}")
print(f"Skills: {[s.description for s in remote_a2a_agent_card.skills]}")
print(f"Examples: {[s.examples for s in remote_a2a_agent_card.skills][0]}")

### Query the remote A2A agent

Our agent is now live on Vertex AI! Let's interact with it.

Agent Engine and its A2A integration provides multiple ways to connect, catering to different developer needs and use cases. We'll explore three common methods:

*   Via Vertex AI SDK for Python
*   Via A2A Client
*   Via http request


#### Via Vertex AI SDK for Python

For Python developers, this is the simplest method. The `remote_a2a_agent` acts as a smart client or proxy that knows how to communicate with the deployed endpoint. This allows you to use the same methods you used for local testing to interact with the remote agent.


##### Send a message to start a task

Again, the code is nearly identical to our local test. We call `on_message_send` with our question. The SDK sends the request to the deployed agent, which kicks off the task on the agent engine. The response contains the `task_id` for our remote job.


In [ ]:
# Create a message
message_data = {
    # Unique ID for this message (for tracking)
    "messageId": f"msg-{os.urandom(8).hex()}",
    # Role identifies the sender (user vs agent)
    "role": "user",
    # The actual message content
    # Parts can include text, files, or structured data
    "parts": [{"kind": "text", "text": "What are the ingredients for a Margarita?"}],
}

# Invoke the agent
response = await remote_a2a_agent.on_message_send(**message_data)

In [ ]:
# The response contains a Task object with status and ID
task_object = None
for chunk in response:
    # Assuming the first chunk contains the task object
    if isinstance(chunk, tuple) and len(chunk) > 0 and hasattr(chunk[0], "id"):
        task_object = chunk[0]
        break

if task_object:
    task_id = task_object.id
    print(f"Task started: {task_id}")
    print(f"Status: {task_object.status.state}")
else:
    print("Could not retrieve the task object from the response.")

##### Get the response

Using the `task_id` from the previous step, we call `on_get_task`.

The SDK polls the Agent Engine endpoint and retrieves the completed task, including the final answer in the artifacts field. We have successfully communicated with our deployed A2A agent.

> **Note**: Running this cell might require few seconds depending on the use case.

In [ ]:
task_data = {
    "id": task_id,
    "historyLength": 1,  # Include conversation history
}

# Get the task result
result = await remote_a2a_agent.on_get_task(**task_data)

In [ ]:
# Artifacts contain the actual results
for artifact in result.artifacts:
    # Access the text through the 'root' attribute of the Part object
    if (
        artifact.parts
        and hasattr(artifact.parts[0], "root")
        and hasattr(artifact.parts[0].root, "text")
    ):
        display(Markdown(f"**Answer**:\n {artifact.parts[0].root.text}"))
    else:
        print("Could not extract text from artifact parts.")

##### (Optional) Cancel a task

If your remote agent executes long run operations, you can always cancel the associated task as shown below:

```py
# Remote agent
task_data ={
    "id":task_id,
}
response = await remote_agent.on_cancel_task(**task_data)
```


#### Via A2A Client

This method is for developers who want to use the standard, open-source a2a-sdk client directly. This is useful if you're building an application that needs to talk to various A2A agents, not just those hosted on Agent Engine, or if you prefer to work directly with the protocol's native objects.


##### Initialize A2A Client

Here, we set up the A2A SDK `ClientFactory`.

We start from the `AgentCard` we fetched from our deployed agent. This is crucial because the client needs the card (especially the url) to know where to send requests.

Next, we get standard Google Cloud authentication credentials.
Then, we create a ClientConfig object, telling it to use standard HTTP transport and providing a httpx client pre-configured with our authentication headers.

Finally, the `factory.create(remote_a2a_agent_card)` call gives us a client instance ready to communicate with our specific agent endpoint.


In [ ]:
# Get authentication token for Google Cloud
bearer_token = get_bearer_token()
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json",
}

# Configure the A2A client factory
# This handles the protocol implementation details
factory = ClientFactory(
    ClientConfig(
        # Specify supported transport mechanisms
        supported_transports=[TransportProtocol.http_json],
        # Use client preferences for protocol negotiation
        use_client_preference=True,
        # Configure HTTP client with authentication
        httpx_client=httpx.AsyncClient(
            timeout = 120,
            headers={
                "Authorization": f"Bearer {bearer_token}",
                "Content-Type": "application/json",
            }
        ),
    )
)


# Create a client for our specific agent
# The client uses the Agent Card to understand capabilities
a2a_client = factory.create(remote_a2a_agent_card)

##### Get the agent card

This is a simple call to the A2A client `get_card()` method to verify that our connection and authentication are configured correctly. It should return the same agent card we've seen before.


In [ ]:
response = await a2a_client.get_card()
print(f"Agent: {remote_a2a_agent_card.name}")
print(f"URL: {remote_a2a_agent_card.url}")
print(f"Skills: {[s.description for s in remote_a2a_agent_card.skills]}")
print(f"Examples: {[s.examples for s in remote_a2a_agent_card.skills][0]}")

##### Send a message to start a task

Once again, we manually construct a `Message` object with a role, parts, and a unique `message_id`. We then call `a2a_client.send_message()`. This method returns an async generator, so we loop through it to get the resulting chunks, which contain the submitted `Task` object. From there, we extract the `task_id`.


In [ ]:
import uuid
taskId = str(uuid.uuid4())
task_id = taskId

In [ ]:
task_id

In [ ]:
# Send a message using A2A protocol objects
message = Message(
    message_id=f"message-{os.urandom(8).hex()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="list a random cocktail"))],
    #task_id=task_id,
)

# Get response
response = a2a_client.send_message(message)

# The response is an async generator
async for response_chunk in response:
    # The response is often a tuple, with the Task as the first element
    task_object = response_chunk[0]
    task_id = task_object.id
print(f"Task started: {task_id}")
print(f"Status: {task_object.status.state}")

##### Get the response

Using the `task_id`, we create a `TaskQueryParams` object and pass it to the `a2a_client.get_task()` method. This fetches the final result from the Agent Engine endpoint, demonstrating a successful interaction using the generic A2A client SDK.

> **Note**: Running this cell might require few seconds depending on the use case.

In [ ]:
task_data = {
    "id": task_id,
    "historyLength": 1,  # Include conversation history
}
response = await a2a_client.get_task(TaskQueryParams(**task_data))

# Check if it has artifacts
if hasattr(response, "artifacts") and response.artifacts:
    for artifact in response.artifacts:
        result_text = artifact.parts[0].root.text
        display(Markdown(f"**Result**\n: {result_text}"))

#### Via http request

This is the most fundamental way to interact with our agent: making direct HTTP requests.

This approach is perfect for debugging with tools like
curl or for integrating from languages that don't have a dedicated A2A or Vertex AI SDK.

##### Get the agent card

To start, we get the endpoint URL from the agent card we fetched earlier. Next, we obtain a bearer token for authentication. Then
we set the necessary Authorization and Content-Type headers. And finally, we use the httpx library to make the GET request and print the resulting JSON response.


In [ ]:
# Prepare authentication headers
headers = {
    "Authorization": f"Bearer {get_bearer_token()}",
    "Content-Type": "application/json",
}

# Get the agent card endpoint
remote_agent_card_url = remote_a2a_agent_card.url
remote_agent_card_endpoint = f"{remote_agent_card_url}/v1/card"

try:
    # Send the HTTP request
    response = httpx.get(remote_agent_card_endpoint, headers=headers)
    response.raise_for_status()
    # Parse the response
    result = response.json()
    print(json.dumps(result, indent=2))
except httpx.HTTPStatusError as e:
    print(f"HTTP error occurred: {e}")
    print(f"Response body: {e.response.text}")
except httpx.RequestError as e:
    print(f"An error occurred while trying to send the request: {e}")

##### Send a message to start a task

Now you can make a POST request to the `/v1/message:send` endpoint. We construct the JSON payload manually, following the structure defined by the A2A protocol. We then send the request using `httpx.post` with the same headers as before. The response is the JSON object for the submitted task, from which we extract the `task_id`.

> **Note**: Running this cell might require few seconds depending on the use case.

In [ ]:
# Construct the A2A message payload
payload = {
    "message": {
        "messageId": f"msg-{os.urandom(8).hex()}",
        "role": "1",  # "1" = user, "2" = agent (in HTTP encoding)
        "content": [{"text": "list a random cocktail"}],
    },
    "metadata": {
        # Optional metadata for tracking/debugging
        "source": "tutorial",
        "timestamp": datetime.now().isoformat(),
        "user_agent": "test_script",
    },
}

try:
    # Send the HTTP request
    response = httpx.post(
        f"{remote_agent_card_url}/v1/message:send", json=payload, headers=headers
    )
    response.raise_for_status()
    # Parse the response
    result = response.json()
    print(json.dumps(result, indent=2))
except httpx.HTTPStatusError as e:
    print(f"HTTP error occurred: {e}")
    print(f"Response body: {e.response.text}")
except httpx.RequestError as e:
    print(f"An error occurred while trying to send the request: {e}")

# The response contains a task
task_id = result["task"]["id"]
task_status = result["task"]["status"]["state"]
print(f"Task started: {task_id}")
print(f"Status: {task_status}")

##### Get the response

Finally, we construct the URL for the specific task using the task_id and make a `GET` request to the `/v1/tasks/{task_id}` endpoint. The response is the full JSON object for the completed task, containing the final answer. This confirms we can interact with our agent using nothing but standard HTTP calls.

In [ ]:
# Poll for completion
task_url = f"{remote_agent_card_url}/v1/tasks/{task_id}"
print(f"Polling for results at: {task_url}")
while True:
    # Poll the task endpoint until it reaches a terminal state.
    response = httpx.get(task_url, headers=headers, params={"history_length": 1})
    response.raise_for_status()
    task_data = response.json()
    state = task_data["status"]["state"]

    if state in ["TASK_STATE_COMPLETED", "TASK_STATE_FAILED"]:
        print(f"Task finished with state: {state}")
        break

    # Wait for a second before checking again to avoid spamming the API.
    time.sleep(1)

# Extract the result
if state == "TASK_STATE_COMPLETED":
    artifacts = task_data.get("artifacts", [])
    for artifact in artifacts:
        for part in artifact["parts"]:
            if "text" in part:
                display(Markdown(f"**Result**\n: {part['text']}"))
                break

## Cleaning up

Time to clean up. Run the cell below prevents you from incurring ongoing costs for the services you've provisioned during this tutorial.

In [ ]:
delete_agent_engine = True

if delete_agent_engine:
    remote_a2a_agent.delete(force=True)